In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import scipy.io as scio
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data_folder = '/cellar/users/majianzhu/xiwen_code/data/'
print(data_folder)

/cellar/users/majianzhu/xiwen_code/data/


# get 1815 genes

In [3]:
data = scio.loadmat(data_folder + 'Demo.mat')

In [4]:
genes = [str(item[0]) for item in data['genenames'].reshape(-1)]

## expression DataFrame

In [5]:
gdsc_exp_df = pd.read_csv(data_folder + 'Cell_line_RMA_proc_basalExp.txt', sep='\t', index_col=0)
gdsc_exp_df = gdsc_exp_df.dropna(how='any')
gdsc_exp_df = gdsc_exp_df.drop('GENE_title', axis=1)
gdsc_exp_df = gdsc_exp_df.rename(columns=lambda x:x[5:])

In [6]:
gdsc_exp_df.head()

,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE_SYMBOLS,,,,,,,,,,,,,,,,,,,,,
TSPAN6,7.632023,7.548671,8.712338,7.797142,7.729268,7.074533,3.285198,6.961606,5.943046,3.455951,...,7.105637,3.236503,3.038892,8.373223,6.932178,8.441628,8.422922,8.089255,3.112333,7.153127
TNMD,2.964585,2.777716,2.643508,2.817923,2.957739,2.889677,2.828203,2.874751,2.686874,3.290184,...,2.798847,2.745137,2.976406,2.852552,2.622630,2.639276,2.879890,2.521169,2.870468,2.834285
DPM1,10.379553,11.807341,9.880733,9.883471,10.418840,9.773987,10.264385,10.205931,10.299757,11.570155,...,10.486486,10.442951,10.311962,10.454830,10.418475,11.463742,10.557777,10.792750,9.873902,10.788218
SCYL3,3.614794,4.066887,3.956230,4.063701,4.341500,4.270903,5.968168,3.715033,3.848112,5.560883,...,3.696835,4.624013,4.348524,3.858121,3.947561,4.425849,3.550390,4.443337,4.266828,4.100493
C1orf112,3.380681,3.732485,3.236620,3.558414,3.840373,3.815055,3.011867,3.268449,3.352835,3.571228,...,3.726833,3.947744,3.806584,3.196988,3.814831,4.384732,4.247189,3.071359,3.230197,3.435795


In [7]:
gdsc_exp_df.tail()

,906826,687983,910927,1240138,1240139,906792,910688,1240135,1290812,907045,...,753584,907044,998184,908145,1659787,1298157,1480372,1298533,930299,905954.1
GENE_SYMBOLS,,,,,,,,,,,,,,,,,,,,,
KRT18P55,3.331134,2.992611,2.886574,3.812119,3.412586,2.999640,2.891768,3.457153,2.965871,3.424035,...,3.257423,3.323432,3.251622,3.041222,3.174122,3.343723,3.412558,3.640390,3.142641,2.927523
NaN,2.852537,2.776771,2.685307,3.436412,2.951270,3.233383,3.810246,2.792116,2.641117,3.124607,...,2.699663,5.190438,3.253381,3.000088,2.846830,2.959009,2.974475,2.903894,2.857956,3.033662
POLRMTP1,3.130696,3.260982,3.176239,3.074432,3.213545,3.382112,3.200106,2.829053,3.158745,3.173933,...,2.773728,2.988250,3.514337,3.254306,3.139208,3.007502,3.088841,2.847505,2.832840,2.817057
UBL5P2,9.986616,9.002814,9.113243,9.958284,9.938978,8.714820,9.396484,9.779745,9.477582,9.518999,...,9.593772,9.506062,9.945730,9.890244,10.018968,9.332193,10.742651,8.544696,9.900550,9.071943
TBC1D3P5,3.073724,3.000182,2.916274,3.256500,3.396126,3.497439,3.193505,3.254539,3.143067,3.240584,...,3.407260,3.256900,3.189972,3.155584,3.357660,3.435411,3.317945,3.174515,3.243563,3.324517


In [8]:
gdsc_exp_df = gdsc_exp_df.loc[genes]

# mutation DataFrame

In [9]:
data = pd.read_csv(data_folder + 'WES_variants.csv', sep='\t')

In [10]:
cell_line_id_nan = []
cell_line_tissue_dic = {}

for name, group in data[['COSMIC_ID', 'Cancer Type']].groupby('COSMIC_ID'):
    name = str(name)
    
    if group['Cancer Type'].nunique() == 1:
        cell_line_tissue_dic[name] = group['Cancer Type'].unique()[0]
    else:
        cell_line_tissue_dic[name] = 'nan'
        cell_line_id_nan.append(name)

In [11]:
gdsc_mut_df = pd.DataFrame()

data = pd.read_csv(data_folder + 'WES_variants.csv', sep='\t')

for index, row in data.iterrows():
    if int(index)%10000 == 0: print(index)
    gdsc_mut_df.loc[row['Gene'], str(row['COSMIC_ID'])] = 1
    
gdsc_mut_df = gdsc_mut_df.fillna(value=0)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000


In [12]:
gdsc_mut_df.head()

,907272,998184,907289,905989,909698,905971,909761,1327769,1330931,1298151,...,908135,1303897,724812,908144,687586,910209,1240132,1290807,1330932,688058
A1BG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CF,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2M,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A2ML1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A4D226,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
gdsc_mut_df.tail()

,907272,998184,907289,905989,909698,905971,909761,1327769,1330931,1298151,...,908135,1303897,724812,908144,687586,910209,1240132,1290807,1330932,688058
ZYG11A,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYG11B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZYX,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZEF1,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ZZZ3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
gdsc_mut_df = gdsc_mut_df.loc[genes]

# compare expression and mutation

In [15]:
cell_line_id_set = set(gdsc_mut_df.columns) & set(gdsc_exp_df.columns)
print (len(cell_line_id_set))

968


# Drug Response DataFrame

In [16]:
gdsc_drug_response_df = pd.read_csv(data_folder + 'v17_fitted_dose_response.csv', sep='\t')

In [17]:
gdsc_drug_response_df.head()

,DATASET_VERSION,IC50_RESULTS_ID,COSMIC_ID,DRUG_ID,MAX_CONC_MICROMOLAR,LN_IC50,AUC,RMSE
0,17,335,924100,1026,1.0,0.717722,0.899410,0.105665
1,17,336,924100,1028,2.0,2.664100,0.957206,0.178243
2,17,337,924100,1029,2.0,3.336828,0.973893,0.079845
3,17,338,924100,1030,10.0,5.164909,0.977844,0.094228
4,17,339,924100,1031,0.2,-4.325309,0.508180,0.090478


In [18]:
drug_id_arr = pd.unique(gdsc_drug_response_df.DRUG_ID)

In [30]:
found_and_unfound_cell_line = {}
gdsc_dataset = {}

In [40]:
for drug_id in [drug_id_arr]:
    found_and_unfound_cell_line[drug_id] = ([],[])
    gdsc_dataset[drug_id] = {
        'exp': pd.DataFrame(columns=genes),
        'mut': pd.DataFrame(columns=genes),
        'auc': pd.Series()
    }
    for index, value in gdsc_drug_response_df.loc[gdsc_drug_response_df.DRUG_ID == drug_id].COSMIC_ID.items():
        value = str(value)
        if value in cell_line_id_set:
            found_and_unfound_cell_line[drug_id][0].append(value)
            gdsc_dataset[drug_id]['exp'].loc[value] = gdsc_exp_df[value]
            gdsc_dataset[drug_id]['mut'].loc[value] = gdsc_mut_df[value]
            gdsc_dataset[drug_id]['auc'].loc[value] = gdsc_drug_response_df.loc[
                (gdsc_drug_response_df['DRUG_ID']==drug_id) & (gdsc_drug_response_df['COSMIC_ID']==int(value))].AUC.item()
        else:
            found_and_unfound_cell_line[drug_id][1].append(value)
for key in gdsc_dataset.keys():
    exp = gdsc_dataset[key]['exp']
    mut = gdsc_dataset[key]['mut']
    gdsc_dataset[key]['mut'] = mut[mut.columns[mut.sum()>4]]
    mut_feature = list(mut.columns[mut.sum()>4])
    thre = exp.var().quantile(0.2)
    gdsc_dataset[key]['exp'] = exp[exp.columns[exp.var()>thre]]

for key in gdsc_dataset.keys():
    exp = gdsc_dataset[key]['exp'].to_numpy().astype(np.float32)
    mut = gdsc_dataset[key]['mut'].to_numpy().astype(np.float32)
    exp = (exp - exp.mean(axis=0))/exp.std(axis=0)
    
    gdsc_dataset[key]['X'] = np.concatenate([exp, mut], axis=1)
    tmp = gdsc_dataset[key]['auc'].to_numpy().reshape(-1,1).astype(np.float32)
    gdsc_dataset[key]['y'] = (tmp-tmp.mean())/tmp.std()
for key in gdsc_dataset.keys():
    idx = list(gdsc_dataset[key]['auc'].argsort())
    for k,v in gdsc_dataset[key].items():
        if k in ['exp','mut','auc']:
            gdsc_dataset[key][k] = v.iloc[idx]
        else:
            gdsc_dataset[key][k] = v[idx]
with open('shuhantao/gdsc_dataset.pickle', 'wb') as f:
    pickle.dump(gdsc_dataset, f)
for key in gdsc_dataset.keys():
        with open(f'./gdsc/data/{key}.pickle', 'wb') as f:
        pickle.dump(tmp, f)